In [1]:
import requests
import urllib
import pandas as pd
import numpy as np

In [2]:
#import and read the csv
dfSat = pd.read_csv("viirs-snpp_2021_United_States.csv") #change this to the FIRMS csv download file name you want
dfSat

,Unnamed: 0,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,0,43.476295,-71.587440,297.23,0.48,0.40,2021-01-01,624,N,VIIRS,n,1,269.23,0.68,N,0
1,1,43.246197,-72.420631,298.24,0.52,0.42,2021-01-01,624,N,VIIRS,n,1,267.55,1.06,N,0
2,2,40.890423,-77.711998,318.41,0.59,0.53,2021-01-01,624,N,VIIRS,n,1,269.67,3.09,N,2
3,3,39.589787,-75.633263,295.77,0.44,0.46,2021-01-01,624,N,VIIRS,n,1,276.95,1.78,N,2
4,4,27.574802,-81.772072,298.91,0.37,0.58,2021-01-01,624,N,VIIRS,n,1,284.31,0.51,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661053,661053,29.796158,-90.769928,333.09,0.40,0.37,2021-12-31,1900,N,VIIRS,n,1,299.65,11.39,D,0
661054,661054,29.784075,-91.766235,326.32,0.42,0.38,2021-12-31,1900,N,VIIRS,n,1,295.23,3.28,D,0
661055,661055,40.144726,-101.530975,342.62,0.51,0.50,2021-12-31,1906,N,VIIRS,n,1,277.59,7.29,D,0
661056,661056,38.554855,-122.550667,334.54,0.52,0.41,2021-12-31,2042,N,VIIRS,n,1,281.30,8.67,D,0


In [3]:
# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# coordinates with known elevation for testing
lat = [43.476295, 43.246197, 40.890423, 39.589787, 27.574802]
log = [-71.587440, -72.420631, -77.711998, -75.633263, -81.772072]


# create data frame
df = pd.DataFrame({
    'latitude': lat,
    'longitude': log
})

def elevation_function(df, lat_column, log_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[log_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations

elevation_function(df, 'latitude', 'longitude')
df.head()

,latitude,longitude,elev_meters
0,43.476295,-71.587440,208.31
1,43.246197,-72.420631,143.97
2,40.890423,-77.711998,322.79
3,39.589787,-75.633263,18.82
4,27.574802,-81.772072,25.89


Only 2000 locations that can be converted at most per request, so randomly selected from the original data as a feature of subsequent data analysis.

ps: I want to do for loop to get more results, please help. Thanks!

In [4]:
# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# select n numbers of rows randomly(maximum is 2000 according to API's rule)
df_sample = dfSat.sample(n=2000)

# create data frame coordinates with known elevation
df = pd.DataFrame({'latitude': df_sample['latitude'],
                   'longitude': df_sample['longitude']})

def elevation_function(df, lat_column, lon_column, block_size=50000):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations
   
elevation_function(df, 'latitude', 'longitude')
df

,latitude,longitude,elev_meters
613274,38.356220,-119.848457,1796.94
566957,39.961861,-76.595161,198.27
155141,41.459930,-83.365303,198.41
178248,36.129890,-113.093422,1797.16
15226,29.833424,-93.962486,1.34
...,...,...,...
552703,36.091908,-118.614418,2041.6
599492,30.787590,-96.676315,94.83
225959,46.158218,-117.461113,1508.15
66725,34.524780,-95.920807,292.48


In [5]:
# Merge the data back to the original DataFrame
df1 = pd.merge(dfSat, df, on = ['latitude','longitude'], how="inner")
df1

,Unnamed: 0,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,elev_meters
0,1218,28.853601,-99.438446,304.03,0.54,0.51,2021-01-04,854,N,VIIRS,n,1,277.73,1.16,N,0,191.24
1,1430,33.013027,-79.884186,337.47,0.41,0.37,2021-01-04,1830,N,VIIRS,n,1,292.87,4.98,D,2,10.29
2,1539,30.858057,-93.781578,327.33,0.52,0.67,2021-01-04,2006,N,VIIRS,n,1,290.42,2.51,D,0,79.59
3,1679,19.412098,-155.271988,343.27,0.39,0.36,2021-01-04,2330,N,VIIRS,n,1,291.18,6.70,D,0,1086.41
4,2339,26.321386,-80.948921,332.73,0.57,0.43,2021-01-05,1806,N,VIIRS,n,1,299.81,6.27,D,0,4.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,659755,32.442856,-101.637222,295.52,0.43,0.46,2021-12-27,900,N,VIIRS,n,1,280.65,0.40,N,0,819.13
1996,659910,36.772472,-120.257133,333.19,0.59,0.53,2021-12-27,2018,N,VIIRS,n,1,282.72,7.96,D,0,49.72
1997,660318,33.544041,-94.297173,314.14,0.48,0.40,2021-12-29,818,N,VIIRS,n,1,286.84,0.96,N,0,94.09
1998,660443,32.854134,-97.879089,347.03,0.40,0.37,2021-12-29,1936,N,VIIRS,n,1,300.11,11.49,D,0,383.76
